In [ ]:
# Ausführen um die Verbindung zur Datenbank aufzubauen

import urllib.request
urllib.request.urlretrieve("https://github.com/tschuegge/SqlKurs/raw/master/connect.py", "connect.py")
def sql(): pass
%run ./connect.py

In [ ]:
# Aufgabe 1.6.1
sql("""

INSERT INTO kunde (kundennr, name, vorname, strasse, plz, ort, telefonprivat, email, zahlungsart)
  VALUES (
    (SELECT MAX(kundennr) + 1 FROM kunde),
    'Iten',
    'Raphaela',
    'Alte Zugerbergstrasse 43',
    '6314',
    'Unterägeri',
    '041 754 23 85',
    'r.iten@bluewin.ch',
    'R'
)

""")

In [ ]:
# Aufgabe 1.6.2
sql("""

-- Die Aufgabe kann mit drei einzelnen Update-Statements oder
-- einer Common Table Expression realisiert werden

-- Variante Update-Statements:
-- UPDATE kunde SET ort = 'Buchs AG' WHERE plz = '5033';
-- UPDATE kunde SET ort = 'Buchs ZH' WHERE plz = '8107';
-- UPDATE kunde SET ort = 'Buchs SG' WHERE plz = '9470';

-- Variante Common Table Expression:
WITH Buchs_CTE(plz, ort) AS (
  VALUES ('5033', 'Buchs AG'),  -- Alternativ können die Werte als SELECT/UNION SELECT angegeben werden
         ('8107', 'Buchs ZH'),
         ('9470', 'Buchs SG')
)
UPDATE kunde SET ort = (SELECT ort FROM Buchs_CTE WHERE plz = kunde.plz)
  WHERE plz in (SELECT plz FROM Buchs_CTE)

""")

In [ ]:
# Aufgabe 1.6.3
sql("""

INSERT INTO kunde(kundennr, name, vorname, strasse, plz, ort, telefongeschaeft, email, zahlungsart)
SELECT
  (SELECT MAX(kundennr) + mitarbeiternr FROM kunde),  -- Da Mitarbeiternummer bei 1 beginnt und aufsteigend ist, eignet sie sich um die Kundennummer zu bilden
  name,
  vorname,
  strasse,
  plz,
  ort,
  telefon,
  email,
  'K'
FROM mitarbeiter

""")

In [ ]:
# Aufgabe 1.6.4
sql("""

-- Zuerst Bestellungspositionen löschen
DELETE FROM bestellungposition WHERE bestellungkopfid IN (SELECT id FROM bestellungkopf WHERE bestelldatum < '2011-01-01');

""")
sql("""

-- Danach die Bestellungsköpfe löschen
DELETE FROM bestellungkopf WHERE bestelldatum < '2011-01-01';

""")

In [ ]:
# Aufgabe 1.6.5
sql("""

WITH PosPrice_CTE(bestellkopfid, price) as (
  SELECT
    p.bestellungkopfid,
    SUM(p.liefermenge * a.nettopreis * (1 + m.satz/100))
  FROM bestellungposition p
    INNER JOIN artikel a ON a.id = p.artikelid
    INNER JOIN mwstsatz m ON m.id = a.mwstid
  GROUP BY p.bestellungkopfid
)
UPDATE bestellungkopf SET rechnungsbetrag = (SELECT price FROM PosPrice_CTE WHERE bestellkopfid = bestellungkopf.id)

""")